In [63]:
#import tensorflow as tf

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from IPython.display import YouTubeVideo
from matplotlib import pyplot as plt

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import tensorflow as tf
import seaborn as sns
#import plotly.plotly as py

import os

In [64]:
print(os.listdir("data/frame"))

['trainrg.tfrecord', 'trainhZ.tfrecord', 'trainXa.tfrecord', 'trainp1.tfrecord', 'trainOh.tfrecord', 'train92.tfrecord', 'traincS.tfrecord']


In [65]:
df = pd.read_csv('vocabularyA.csv')

In [66]:
df.head(2)

,Index,TrainVideoCount,KnowledgeGraphId,Name,WikiUrl,Vertical1,Vertical2,Vertical3,WikiDescription
0,0,788288,/m/03bt1gh,Game,https://en.wikipedia.org/wiki/Game,Games,NaN,NaN,"A game is structured form of play, usually und..."
1,1,539945,/m/01mw1,Video game,https://en.wikipedia.org/wiki/Video_game,Games,NaN,NaN,A video game is an electronic game that involv...


In [67]:
df.isnull().sum()

Index                  0
TrainVideoCount        0
KnowledgeGraphId       0
Name                  56
WikiUrl               56
Vertical1              0
Vertical2           3278
Vertical3           3830
WikiDescription       56
dtype: int64

In [68]:
df['Index'].unique

<bound method Series.unique of 0          0
1          1
2          2
3          3
4          4
        ... 
3857    3805
3858    3854
3859    3824
3860    3844
3861    3812
Name: Index, Length: 3862, dtype: int64>

In [69]:
df['Name'].value_counts()

Wardrobe                      1
Wedding ring                  1
Women's Tennis Association    1
Beauty salon                  1
Mirror                        1
                             ..
Hummer                        1
Rust (video game)             1
Christ (title)                1
Cardboard                     1
Final Fantasy Type-0          1
Name: Name, Length: 3806, dtype: int64

In [70]:
df['Vertical1'].value_counts()

Games                      734
Arts & Entertainment       525
Autos & Vehicles           466
Computers & Electronics    302
Food & Drink               278
Business & Industrial      259
Sports                     167
Pets & Animals             150
(Unknown)                  138
Hobbies & Leisure          132
Home & Garden              120
Science                    116
Shopping                   116
Beauty & Fitness            86
Travel                      48
People & Society            46
Reference                   41
Law & Government            32
Internet & Telecom          26
Books & Literature          20
News                        18
Health                      14
Jobs & Education            12
Finance                     10
Real Estate                  6
Name: Vertical1, dtype: int64

In [71]:
label_names = df[['Index', 'Vertical1']]
label_names.head(2)

,Index,Vertical1
0,0,Games
1,1,Games


In [72]:
label_names.rename(columns={"Index": "label_id", "Vertical1": "label_name"}, inplace=True)

In [73]:
label_names.head(2)

,label_id,label_name
0,0,Games
1,1,Games


In [74]:
frame_files = ["data/frame/{}".format(i) for i in os.listdir("data/frame")]

vid_ids = []
labels = []
feat_rgb = []
feat_audio = []

#extract video ID and labels
for file in frame_files:
    for example in tf.python_io.tf_record_iterator(file):
        tf_example = tf.train.Example.FromString(example)

        vid_ids.append(tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
        labels.append(tf_example.features.feature['labels'].int64_list.value)
        
#extract rgb features and audio features from each video        
for file in frame_files:
    for example in tf.python_io.tf_record_iterator(file):        
        tf_seq_example = tf.train.SequenceExample.FromString(example)
        n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
        sess = tf.InteractiveSession()
        rgb_frame = []
        audio_frame = []
        # iterate through frames
        for i in range(n_frames):
            rgb_frame.append(tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval())
            audio_frame.append(tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval())


        sess.close()
        feat_rgb.append(rgb_frame)
        feat_audio.append(audio_frame)
        break

In [75]:
print("No. of videos %d" % len(feat_rgb))
print('The first video has %d frames' %len(feat_rgb[0]))
print("Max frame length is: %d" % max([len(x) for x in feat_rgb]))
print('\n')
print('Picking a youtube video id: %s' % vid_ids[1])
print('List of label ids for youtube video id %s, are - %s' % (vid_ids[1], str(labels[1])))
print('\n')
print('First 20 rgb feature of a youtube video (',vid_ids[1],'): are %s' % str(feat_rgb[1][:20]))


No. of videos 7
The first video has 280 frames
Max frame length is: 300


Picking a youtube video id: Fmrg
List of label ids for youtube video id Fmrg, are - [39, 648]


First 20 rgb feature of a youtube video ( Fmrg ): are [array([ 92., 115.,  73., ...,  88., 114.,  81.], dtype=float32), array([145.,  92.,  53., ..., 102., 119., 184.], dtype=float32), array([162.,  74.,  52., ..., 163., 120., 178.], dtype=float32), array([106.,  69.,  50., ..., 161., 186., 112.], dtype=float32), array([ 84., 146.,  62., ..., 177.,  71., 153.], dtype=float32), array([145.,  51.,  40., ..., 158., 203., 113.], dtype=float32), array([138.,  83.,  49., ..., 209., 142.,  68.], dtype=float32), array([146.,  57.,  58., ..., 181., 255.,  85.], dtype=float32), array([ 53., 143.,  67., ..., 194., 162.,  61.], dtype=float32), array([ 97.,  52.,  48., ...,  86., 255.,   0.], dtype=float32), array([107., 114.,  76., ..., 199., 188., 137.], dtype=float32), array([101., 100.,  85., ..., 167.,  97., 155.], dtype=float

In [76]:
#identify labels for specific video

labels_name = []

for row in labels:
    n_labels = []
    for label_id in row:
        # some labels ids are missing so have put try/except
        try:
            n_labels.append(str(label_names[label_names['label_id']==label_id]['label_name'].values[0]))
        except:
            continue
    labels_name.append(n_labels)

print('List of label names for youtube video id %s, are - %s' % (vid_ids[1], str(labels_name[1])))

List of label names for youtube video id Fmrg, are - ['Beauty & Fitness', 'Shopping']


In [ ]:
### MODELING PART

In [78]:
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model
import operator
import time 
import gc


Using TensorFlow backend.


In [80]:
def create_train_dev_dataset(frame_rgb, frame_audio, labels):
    """
    Method to created training and validation data
    """
    shuffle_indices = np.random.permutation(np.arange(len(labels)))
    #video_rgb_shuffled = video_rgb[shuffle_indices]
    #video_audio_shuffled = video_audio[shuffle_indices]
    frame_rgb_shuffled = frame_rgb[shuffle_indices]
    frame_audio_shuffled = frame_audio[shuffle_indices]
    labels_shuffled = labels[shuffle_indices]

    dev_idx = max(1, int(len(labels_shuffled) * validation_split_ratio))

    #del video_rgb
    #del video_audio
    del frame_rgb
    del frame_audio
    gc.collect()

    #train_video_rgb, val_video_rgb = video_rgb_shuffled[:-dev_idx], video_rgb_shuffled[-dev_idx:]
    #train_video_audio, val_video_audio = video_audio_shuffled[:-dev_idx], video_audio_shuffled[-dev_idx:]
    
    train_frame_rgb, val_frame_rgb = frame_rgb_shuffled[:-dev_idx], frame_rgb_shuffled[-dev_idx:]
    train_frame_audio, val_frame_audio = frame_audio_shuffled[:-dev_idx], frame_audio_shuffled[-dev_idx:]
    
    train_labels, val_labels = labels_shuffled[:-dev_idx], labels_shuffled[-dev_idx:]
    
    del frame_rgb_shuffled, frame_audio_shuffled, labels_shuffled
    gc.collect()
    
    return (train_frame_rgb, train_frame_audio, train_labels, 
            val_frame_rgb, val_frame_audio, val_labels)

In [81]:
max_frame_rgb_sequence_length = 10
frame_rgb_embedding_size = 1024

max_frame_audio_sequence_length = 10
frame_audio_embedding_size = 128

number_dense_units = 1000
number_lstm_units = 100
rate_drop_lstm = 0.2
rate_drop_dense = 0.2
activation_function='relu'
validation_split_ratio = 0.2
label_feature_size = 10


def create_model(frame_rgb, frame_audio, labels):
#def create_model(video_rgb, video_audio, frame_rgb, frame_audio, labels):
    """Create and store best model at `checkpoint` path ustilising bi-lstm layer for frame level data of videos"""
    #train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio, train_labels, val_video_rgb, val_video_audio, val_frame_rgb, val_frame_audio, val_labels = create_train_dev_dataset(video_rgb, video_audio, frame_rgb, frame_audio, labels) 
    train_frame_rgb, train_frame_audio, train_labels, val_frame_rgb, val_frame_audio,val_labels = create_train_dev_dataset(frame_rgb, frame_audio, labels) 
    
    
    # Creating 2 bi-lstm layer, one for rgb and other for audio level data
    lstm_layer_1 = Bidirectional(LSTM(number_lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
    lstm_layer_2 = Bidirectional(LSTM(number_lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
    
    # creating input layer for frame-level data
    frame_rgb_sequence_input = Input(shape=(max_frame_rgb_sequence_length, frame_rgb_embedding_size), dtype='float32')
    frame_audio_sequence_input = Input(shape=(max_frame_audio_sequence_length, frame_audio_embedding_size), dtype='float32')
    
    frame_x1 = lstm_layer_1(frame_rgb_sequence_input)
    frame_x2 = lstm_layer_2(frame_audio_sequence_input)
    
    # creating input layer for video-level data
    #video_rgb_input = Input(shape=(video_rgb.shape[1],))
    #video_rgb_dense = Dense(int(number_dense_units/2), activation=activation_function)(video_rgb_input)
    
    #video_audio_input = Input(shape=(video_audio.shape[1],))
    #video_audio_dense = Dense(int(number_dense_units/2), activation=activation_function)(video_audio_input)
    
    # merging frame-level bi-lstm output and later passed to dense layer by applying batch-normalisation and dropout
    merged_frame = concatenate([frame_x1, frame_x2])
    merged_frame = BatchNormalization()(merged_frame)
    merged_frame = Dropout(rate_drop_dense)(merged_frame)
    merged_frame_dense = Dense(int(number_dense_units/2), activation=activation_function)(merged_frame)
    
    # merging video-level dense layer output
#     merged_video = concatenate([video_rgb_dense, video_audio_dense])
#     merged_video = BatchNormalization()(merged_video)
#     merged_video = Dropout(rate_drop_dense)(merged_video)
#     merged_video_dense = Dense(int(number_dense_units/2), activation=activation_function)(merged_video)
    
    # merging frame-level and video-level dense layer output
    merged = concatenate([merged_frame_dense, merged_video_dense])
    merged = BatchNormalization()(merged)
    merged = Dropout(rate_drop_dense)(merged)
     
    merged = Dense(number_dense_units, activation=activation_function)(merged)
    merged = BatchNormalization()(merged)
    merged = Dropout(rate_drop_dense)(merged)
    preds = Dense(label_feature_size, activation='sigmoid')(merged)
    
    model = Model(inputs=[frame_rgb_sequence_input, frame_audio_sequence_input], outputs=preds)
    print(model.summary())
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    
    STAMP = 'lstm_%d_%d_%.2f_%.2f' % (number_lstm_units, number_dense_units, rate_drop_lstm, rate_drop_dense)
    
    checkpoint_dir = 'checkpoints/' + str(int(time.time())) + '/'

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    bst_model_path = checkpoint_dir + STAMP + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)
    tensorboard = TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))
    
    model.fit([train_frame_rgb, train_frame_audio], train_labels,
              validation_data=([val_frame_rgb, val_frame_audio], val_labels),
              epochs=200, batch_size=64, shuffle=True, callbacks=[early_stopping, model_checkpoint, tensorboard])    
    
    return model
    
    

In [82]:
import random

sample_length = 1000

#video_rgb = np.random.rand(sample_length, 1024)
#video_audio = np.random.rand(sample_length, 128)

frame_rgb = np.random.rand(sample_length, 10, 1024)
frame_audio = np.random.rand(sample_length, 10, 128)

# Here I have considered i have only 10 labels
labels = np.zeros([sample_length,10])
for i in range(len(labels)):
    j = random.randint(0,9)
    labels[i][j] = 1 

In [83]:
model = create_model(frame_rgb, frame_audio, labels)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


RuntimeError: Attempting to capture an EagerTensor without building a function.

In [84]:
# testing w random test data

test_video_rgb = np.random.rand(1, 1024)
test_video_audio = np.random.rand(1, 128)

test_frame_rgb = np.random.rand(1, 10, 1024)
test_frame_audio = np.random.rand(1, 10, 128)

preds = list(model.predict([test_frame_rgb, test_frame_audio, test_video_rgb, test_video_audio], verbose=1).ravel())
index, value = max(enumerate(preds), key=operator.itemgetter(1))
print("Predicted Label - %s with probability - %s" % (str(index), str(value)))

NameError: name 'model' is not defined